In [8]:
import pandas as pd

In [13]:
df = pd.read_parquet('tiling_results.parquet')

In [14]:
df.shape

(94, 12)

In [15]:
len(df[df.tiling_compatible == True])

30

In [16]:
df.concept_id[0]

'G2189788741-LPCLOUD'